In [ ]:
import pandas as pd
df_sample = pd.read_csv("/Users/htg/Desktop/Sydney_University/冒犯性检测/dataset/data_emnlp.csv", sep=",")

In [ ]:
#Chatgpt 4o CPP
import pandas as pd
from openai import OpenAI

# 读取数据集
texts = df_sample.iloc[:, 0]
pinyin = []
api_key ="Your own key"
client = OpenAI(api_key=api_key)
for idx, line in enumerate(texts):
    prompt = (
    "将所有文本按照中文发音转换为不带声调的汉语拼音。\n"
    "请只输出不带声调拼音,不要输出其他内容，输出内容不要换行\n"
    f"文本如下：\n{line}\n"
    )   

    stream = client.chat.completions.create(
    model="gpt-4o-2024-11-20",
    messages=[{"role": "user", "content": prompt}],
    max_completion_tokens=5000,
    temperature=0.1,
    stop=None,  
    stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    print(f"第 {idx+1} 条数据拼音：{result.strip()}")
    pinyin.append(result)


In [ ]:
results = []

df = pd.DataFrame({
    "text": texts,
    "pinyin": pinyin
})

# 合并成一个新列
df["combined"] = df["text"] + " | " + df["pinyin"]
text = df["combined"]

for idx, line in enumerate(text):
    prompt = (
    "你是一位服务中文社交媒体的评论审核专家。\n"
    "请结合下面的汉语拼音和文本判断是否具有冒犯性\n"
    "这里的冒犯性主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。\n" 
    "如果文本具有冒犯性，请仅回复数字 1；如果不具有冒犯性，请仅回复数字 0。\n"
    f"文本如下：\n{line}\n"
    
    )   

    stream = client.chat.completions.create(
    model="gpt-4o-2024-11-20",
    messages=[{"role": "user", "content": prompt}],
    max_completion_tokens=5000,
    temperature=0.1,
    stop=None,  
    stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    print(f"第 {idx+1} 条数据判断结果：{result.strip()}")
    results.append(result.strip())

# 保存结果
result_df = pd.DataFrame({
    "Content": texts,
    "Result": results
})

result_df.to_csv("./Pinyin_result/chatgpt_4o_results.csv", sep="\t", index=False)